<a href="https://colab.research.google.com/github/ThanuMahee12/ayush-vision/blob/yolo/Algorithums/detection/Yolo/YOLO_FLOWER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YOLO Flower

## Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
basicdata={'account':'sanashajeeva@gmail.com','user':"Yuka"}


In [ ]:
! locale-gen en_US.UTF-8
! export LC_ALL=en_US.UTF-8

Generating locales (this might take a while)...
  en_US.UTF-8... done
Generation complete.


## Intallations

In [ ]:
!pip install firebase-admin

In [ ]:
!pip install ultralytics

## Imports

In [ ]:
import firebase_admin


In [ ]:
from firebase_admin import credentials,storage


In [ ]:
from firebase_admin import firestore

In [ ]:
from ultralytics import YOLO

In [ ]:
from datetime import date,datetime

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
import random

In [ ]:
import yaml

In [ ]:
import csv

In [ ]:
import json

## Config

In [ ]:
epochs=10
imageSize=640
optimizer='Adam'
dropout=0.1

### Dirctory

In [ ]:
work_dirctory="/content/drive/MyDrive/AyushVision/yolo8/training"
work_dirctory_root="/content/drive/MyDrive/AyushVision/yolo8"
firebase_key="/content/drive/MyDrive/AyushVision/ServiceKey.json"

In [ ]:
today_work_dirctory=f"{work_dirctory}/{date.today().strftime('%d_%m_%Y')}"

In [ ]:
if not os.path.exists(work_dirctory):
  os.makedirs(work_dirctory,exist_ok=True)
if not os.path.exists(today_work_dirctory):
  os.makedirs(today_work_dirctory,exist_ok=True)

In [ ]:
data="/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/data.yaml"

In [ ]:
project=f"{today_work_dirctory}/flowers/runs"
name=f'train{epochs}'

## Firebase

In [ ]:
cred = credentials.Certificate(firebase_key)
firebase_admin.initialize_app(cred,{
    'storageBucket': 'ayush-vision-asw4gh.appspot.com'
})

In [ ]:
db = firestore.client()

In [ ]:
bucket = storage.bucket()

In [ ]:
yolo_ref = db.collection("training").document('yolo')
doc = yolo_ref.get()

In [ ]:
if not doc.exists:
  doc_ref= db.collection("training").document('yolo')
  doc_ref.set({'algorithum':'yolo','version':'V8','mode':'detection','url':'ultralytics'})
else:
  doc_ref= yolo_ref

### Roboflow

## Model

In [ ]:
today_ref=doc_ref.collection("Flower").document(datetime.now().strftime(f"%Y%m%d%H"))

In [ ]:
basicdata['imageSize']=imageSize
basicdata['project']=project
basicdata['name']=name
basicdata['training_date']=datetime.now().strftime("%Y/%m/%d %H:%M:%S")
basicdata['filename']='YOLO_FLOWER'

In [ ]:
with open(data, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [ ]:
basicdata['names']=yaml_content['names']

In [ ]:
today_ref.set(basicdata)

update_time {
  seconds: 1723365496
  nanos: 394311000
}

In [ ]:
yolo_model = YOLO('yolov8n.yaml').load('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 318MB/s]

Transferred 355/355 items from pretrained weights


#### Training

In [35]:
results = yolo_model.train(data=data,name=name, epochs=epochs, imgsz=imageSize,project=project,optimizer=optimizer,plots=True,dropout=dropout)

Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs, name=train10, exist_ok=False, pretrained=yolov8n.pt, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e

100%|██████████| 755k/755k [00:00<00:00, 22.8MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

train: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/train/labels... 4620 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4620/4620 [29:54<00:00,  2.57it/s]


train: New cache created: /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

      10/10      2.28G       1.58      1.682      1.826         27        640: 100%|██████████| 289/289 [01:57<00:00,  2.45it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:06<00:00,  2.26it/s]

                   all        441        683      0.692      0.631      0.691      0.399



10 epochs completed in 0.354 hours.
Optimizer stripped from /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train10/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train10/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train10/weights/best.pt...
Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 14/14 [00:12<00:00,  1.16it/s]


                   all        441        683      0.693       0.63      0.691        0.4
        Acalyphaindica         40        136      0.661     0.0956      0.342      0.142
              Hibiscus         40         50      0.846      0.876      0.919      0.567
             Holybasil         40         43      0.815      0.837      0.875      0.536
        Pedalium_murex         41         85      0.649      0.635      0.657      0.418
      Red_Pea_Eggplant         39         39      0.684      0.887      0.901      0.645
        Ringworm_plant         40         44       0.49      0.546      0.549      0.325
          Sleepy_plant         40         41      0.888      0.963      0.957      0.557
           Thorn_apple         40         49      0.852      0.755      0.871       0.52
          Vitexnegundo         40         52      0.529      0.519      0.609      0.253
       adamant_creeper         40         52      0.497      0.513       0.49       0.22
   calotropis_gigante

In [36]:
args_yaml=f'{project}/{name}/args.yaml'

In [37]:
resultcsv=f'{project}/{name}/results.csv'

In [38]:
with open(args_yaml, 'r') as file:
    yaml_content = yaml.safe_load(file)

In [39]:
today_training_colref=today_ref.collection('Train')

In [40]:
today_training_ref=today_training_colref.document('args')

In [41]:
today_training_ref.set(yaml_content)

update_time {
  seconds: 1724215738
  nanos: 359528000
}

In [42]:
todayresultref=today_training_colref.document('results')

In [ ]:
todayresultref.set({'results':data_dict})

In [43]:
trainfiles={}
wights={}

In [44]:
for file in os.listdir(f"{project}/{name}"):
  if os.path.isfile(f"{project}/{name}/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/{name}/{file}")
    blob.make_public()
    trainfiles[os.path.splitext(file)[0]]=blob.public_url


In [45]:
for file in os.listdir(f"{project}/{name}/weights"):
  if os.path.isfile(f"{project}/{name}/weights/{file}"):
    blob = bucket.blob(f"yolo/flower/train/{epochs}/weights/{file}")
    blob.upload_from_filename(f"{project}/{name}/weights/{file}")
    blob.make_public()
    wights[os.path.splitext(file)[0]]=blob.public_url

In [46]:
today_training_files=today_training_colref.document('files')
today_training_files.set(trainfiles)

update_time {
  seconds: 1724215770
  nanos: 48837000
}

In [47]:
today_weights_files=today_training_colref.document('weights')
today_weights_files.set(wights)

update_time {
  seconds: 1724215770
  nanos: 641287000
}

#### Validation

In [48]:
model_train_path=f'{project}/{name}/weights/best.pt'

In [49]:
model = YOLO(model_train_path)  # load a custom trained model

In [50]:
metrics = model.val(save_json=True,plots=True,project=project,name=f'val{epochs}')  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n summary (fused): 168 layers, 3,007,988 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/valid/labels.cache... 441 images, 1 backgrounds, 0 corrupt: 100%|██████████| 441/441 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:11<00:00,  2.36it/s]


                   all        441        683      0.693      0.632      0.691        0.4
        Acalyphaindica         40        136      0.659     0.0956      0.341      0.141
              Hibiscus         40         50      0.846      0.877      0.919      0.568
             Holybasil         40         43      0.814      0.837      0.874      0.535
        Pedalium_murex         41         85      0.648      0.635      0.657      0.418
      Red_Pea_Eggplant         39         39      0.684      0.888      0.901      0.645
        Ringworm_plant         40         44      0.493      0.553       0.55      0.325
          Sleepy_plant         40         41      0.888      0.964      0.957      0.555
           Thorn_apple         40         49      0.851      0.755      0.871       0.52
          Vitexnegundo         40         52      0.532      0.525      0.611      0.253
       adamant_creeper         40         52      0.498      0.516      0.492      0.223
   calotropis_gigante

In [51]:
val_today_col=today_ref.collection('Val')

In [52]:
predication_file=f'{project}/val{epochs}/predictions.json'

In [53]:
with open(predication_file, 'r') as file:
    data_dict = json.load(file)

In [54]:
val_today_ref_files=val_today_col.document('files')

In [55]:
valfiles={}

In [56]:
for file in os.listdir(f"{project}/val{epochs}"):
  if os.path.isfile(f"{project}/val{epochs}/{file}"):
    blob = bucket.blob(f"yolo/flowers/val/{epochs}/{file}")
    blob.upload_from_filename(f"{project}/val{epochs}/{file}")
    blob.make_public()
    valfiles[os.path.splitext(file)[0]]=blob.public_url

In [57]:
val_today_ref_files.set(valfiles)

update_time {
  seconds: 1724216522
  nanos: 733066000
}

In [58]:
val_today_ref_metrics=val_today_col.document('metricsbox')

In [59]:
val_today_ref_metrics.set({'map50':metrics.box.map50,'map75':metrics.box.map75})

update_time {
  seconds: 1724216522
  nanos: 912932000
}

In [60]:
val_today_ref_metrics.update({
    'fitness':metrics.fitness
})

update_time {
  seconds: 1724216523
  nanos: 498464000
}

In [61]:
maps=np.array(metrics.maps).tolist()
boxmaps=np.array(metrics.box.maps).tolist()
p=np.array(metrics.box.p).tolist()

In [62]:
val_today_ref_metrics.update({
    'metricsmaps':maps,
    'maps':boxmaps,
    'p':p
})

update_time {
  seconds: 1724216523
  nanos: 665419000
}

In [63]:
metrics.box.map50

0.6914307793840421

In [64]:
metrics.box.map75

0.4013959989126352

In [65]:
metrics.box.maps

array([    0.14064,     0.56759,     0.53507,     0.41798,     0.64497,     0.32543,     0.55523,     0.52001,     0.25337,     0.39993,     0.22271,     0.21619])

#### Predict

In [68]:

results=model.predict(source=f'{work_dirctory_root}/dataset/ayushvision_flowers/test/images',save=True)


image 1/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0012_jpg.rf.90b4658954b2a9b3f1305f3956af7752.jpg: 640x640 1 Holybasil, 8.0ms
image 2/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0169_jpg.rf.582654067138f7912b75dc4648133b1c.jpg: 640x640 1 adamant_creeper, 7.6ms
image 3/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0231_jpg.rf.ad6ab6e527e0375e058347ce9d72b4db.jpg: 640x640 1 Holybasil, 22.8ms
image 4/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0232_jpg.rf.7307ae7cb6640bf4f420042ff633db08.jpg: 640x640 1 Holybasil, 7.8ms
image 5/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushvision_flowers/test/images/IMG-20240121-WA0246_jpg.rf.9e1d4eaab0d82565149c123dae8a477c.jpg: 640x640 1 Holybasil, 7.5ms
image 6/219 /content/drive/MyDrive/AyushVision/yolo8/dataset/ayushv

In [69]:
predicts=[]

In [ ]:
for file in os.listdir(results[0].save_dir):
  if os.path.isfile(f"{results[0].save_dir}/{file}"):
    blob = bucket.blob(f"yolo/flower/predict/{epochs}/{file}")
    blob.upload_from_filename(f"{results[0].save_dir}/{file}")
    blob.make_public()
    predicts.append(blob.public_url)

160

In [74]:
today_ref.collection('Predict').add({
    'files':predicts
})

(DatetimeWithNanoseconds(2024, 8, 21, 5, 7, 17, 176287, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7c835d685e40>)

### Export Model

In [75]:
format='onnx'
model.export(format=format,keras=True,dynamic=True,simplify=True,)

Ultralytics YOLOv8.2.79 🚀 Python-3.10.12 torch-2.3.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from '/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train10/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 16, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnxslim>=0.1.31', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 253.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 214.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.1/223.1 MB 219.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 179.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 200.7 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 20.2s, installed 3 packages: ['onnx>=1.12.0', 'onnxslim>=0.1.31', 'onnxruntime-gpu']
requirements: ⚠️ Restart runtime or rerun command for 

'/content/drive/MyDrive/AyushVision/yolo8/training/21_08_2024/flowers/runs/train10/weights/best.onnx'

In [77]:
blob = bucket.blob(f"yolo/flowers/model/{epochs}/best.onnx")
blob.upload_from_filename(f"{today_work_dirctory}/flowers/runs/{name}/weights/best.onnx")
blob.make_public()

In [78]:
today_ref.collection('Model').add({
    'format':format,
    "keras":True,
    "dynamic":True,
    "simplify":True,
    "model":blob.public_url
})

(DatetimeWithNanoseconds(2024, 8, 21, 5, 9, 40, 976169, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7c835d685cc0>)